<a href="https://colab.research.google.com/github/studentsept/26/blob/main/Copy_of_PRO_C115_Reference_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the Dataset

In [ ]:
#Load the dataset from given directory
!git clone https://github.com/procodingclass/PRO-C114-Text-Sentiment-Dataset

Cloning into 'PRO-C114-Text-Sentiment-Dataset'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (20/20), done.


## Pandas:
**Pandas** is an open-source library built on top of Numpy and Matplotlib. It provides high-performance, easy-to-use data structures and data analysis tools.

A DataFrame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

Pandas DataFrame will be created by loading the datasets from existing MS Excel files, CSV files or SQL Database. Pandas DataFrame can also be created from the lists, dictionaries etc.


## Use Pandas to display the dataset


In [ ]:
import pandas as pd

#read excel file
train_data_raw = pd.read_excel("/content/PRO-C114-Text-Sentiment-Dataset/text-emotion-training-dataset.xlsx")

#display first five entries of training dataset
train_data_raw.head()

,Text_Emotion
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


## Split the rows in two columns as Text and Emotions

In [ ]:
#Split the rows in two columns as Text and Emotions
train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(), 
                                                     columns = ['Text','Emotion'])

In [ ]:
train_data.head(200)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
195,i died would alex and matt feel regretful for ...,sadness
196,i feel that educating families and supporting ...,joy
197,im sure its because when i am lost i feel like...,anger
198,i feel like these are very boring sewing makes...,sadness


## Giving labels to Emotions

In [ ]:
#Find unique emotions
train_data["Emotion"].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
#Create a Dictionary to replace emotions with labels
encode_emotions = {"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}

In [ ]:
train_data.replace(encode_emotions, inplace = True)
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


## Convert Dataframe to list of dataset

In [ ]:
# Convert Dataframe to list of dataset

training_sentences = []
training_labels = []

# append text and emotions in the list using the 'loc' method

for i in range(len(train_data)):
  
  sentence = train_data.loc[i, "Text"]
  training_sentences.append(sentence)

  label = train_data.loc[i, "Emotion"]
  training_labels.append(label)

In [ ]:
#Check a random text and label of the list

training_sentences[50], training_labels[50]

('i need to feel the dough to make sure its just perfect', 2)

## Tokenization & Padding

The act of converting text into numbers is known as **Tokenization**. The **Tokenizer** class of Keras is used for encoding text input into integer sequence. 

**Padding** It is important to make all the sentences contain the same number of words. Zero is used for padding the tokenized sequence to make text contain the same number of tokens. 


In [ ]:
#import Tokenizer from tensorflow

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

#Define parameters for Tokenizer

vocab_size = 10000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

#Create a word_index dictionary

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)

training_sequences[0:2]


[[2, 139, 3, 679],
 [2,
  40,
  101,
  60,
  8,
  15,
  494,
  5,
  15,
  3496,
  553,
  32,
  60,
  61,
  128,
  148,
  76,
  1480,
  4,
  22,
  1255]]

In [ ]:
print(training_sequences[0])
print(training_sequences[1])
print(training_sequences[2])

[2, 139, 3, 679]
[2, 40, 101, 60, 8, 15, 494, 5, 15, 3496, 553, 32, 60, 61, 128, 148, 76, 1480, 4, 22, 1255]
[17, 3060, 7, 1149, 5, 286, 2, 3, 495, 438]


In [ ]:
#Define parameters for pad_sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type='post'
max_length = 100
trunc_type='post'


training_padded = pad_sequences(training_sequences, maxlen=max_length, 
                                padding=padding_type, truncating=trunc_type)

training_padded

array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)

## Converting padded sequences and labels into an Numpy array

In [ ]:

import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

## Model Compilation


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D

model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(0.2),

        Conv1D(filters = 256, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),

        Conv1D(filters = 128, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),

        LSTM(128),

        Dense(128, activation = "relu"),
        Dropout(0.2),
        Dense(64, activation = "relu"),
        Dense(6, activation = "softmax")])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## Model Summary

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 256)           12544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 32, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 10, 128)          0         
 1D)                                                    

## Train Model

In [ ]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, verbose=2)

Epoch 1/30
500/500 - 20s - loss: 0.1356 - accuracy: 0.9536 - 20s/epoch - 39ms/step
Epoch 2/30
500/500 - 20s - loss: 0.1279 - accuracy: 0.9559 - 20s/epoch - 39ms/step
Epoch 3/30
500/500 - 20s - loss: 0.1156 - accuracy: 0.9603 - 20s/epoch - 39ms/step
Epoch 4/30
500/500 - 20s - loss: 0.1039 - accuracy: 0.9628 - 20s/epoch - 40ms/step
Epoch 5/30
500/500 - 19s - loss: 0.0986 - accuracy: 0.9650 - 19s/epoch - 39ms/step
Epoch 6/30
500/500 - 21s - loss: 0.0876 - accuracy: 0.9689 - 21s/epoch - 42ms/step
Epoch 7/30
500/500 - 20s - loss: 0.0910 - accuracy: 0.9694 - 20s/epoch - 40ms/step
Epoch 8/30
500/500 - 20s - loss: 0.0842 - accuracy: 0.9696 - 20s/epoch - 40ms/step
Epoch 9/30
500/500 - 20s - loss: 0.0792 - accuracy: 0.9725 - 20s/epoch - 39ms/step
Epoch 10/30
500/500 - 20s - loss: 0.0721 - accuracy: 0.9759 - 20s/epoch - 39ms/step
Epoch 11/30
500/500 - 20s - loss: 0.0691 - accuracy: 0.9769 - 20s/epoch - 39ms/step
Epoch 12/30
500/500 - 19s - loss: 0.0689 - accuracy: 0.9761 - 19s/epoch - 39ms/step
E

## Save Model

In [ ]:
model.save("Text_Emotion.h5")

## Test the model

In [ ]:
sentence = ["I am happy to meet my friends. We are planning to go to a party.", 
            "I had a bad day at school. i got hurt while playing football"]

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, maxlen=max_length, 
                       padding=padding_type, truncating=trunc_type)

result=model.predict(padded)

predict_class = np.argmax(result, axis=1)
predict_class

# {"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}

array([0, 4])